In [2]:
using TrajectoryOptimization
using RobotDynamics
# using RobotZoo: Cartpole
using StaticArrays, LinearAlgebra

In [3]:
using ForwardDiff
using FiniteDiff


RobotDynamics.@autodiff struct Cartpole{T} <: RobotDynamics.ContinuousDynamics
    mc::T
    mp::T
    l::T
    g::T
end


RobotDynamics.state_dim(::Cartpole) = 4
RobotDynamics.control_dim(::Cartpole) = 1

function RobotDynamics.dynamics(model::Cartpole, x, u)
    mc = model.mc   # mass of the cart in kg (10)
    mp = model.mp   # mass of the pole (point mass at the end) in kg
    l = model.l     # length of the pole in m
    g = model.g     # gravity m/s^2

    q  = x[ SA[1,2] ]  # SA[...] creates a StaticArray.
    qd = x[ SA[3,4] ]

    s = sin(q[2])
    c = cos(q[2])

    H = @SMatrix [mc+mp mp*l*c; mp*l*c mp*l^2]
    C = @SMatrix [0 -mp*qd[2]*l*s; 0 0]
    G = @SVector [0, mp*g*l*s]
    B = @SVector [1, 0]

    qdd = -H\(C*qd + G - B*u[1])
    return [qd; qdd]
end

# function RobotDynamics.dynamics!(model::Cartpole, xdot, x, u)
#     xstatic = SA[x[1], x[2], x[3], x[4]]
#     ustatic = SA[u[1]]
#     xdot .= RobotDynamics.dynamics(model, xstatic, ustatic)
#     return nothing
# end

Cartpole() = Cartpole{Float64}(1.0, 0.2, 0.5, 9.81)

Cartpole

In [4]:
# Dynamics and Constants
model = Cartpole()
n,m = RobotDynamics.dims(model)
N = 101   # number of knot points
tf = 5.0  # final time
x0 = @SVector [0, 0, 0, 0.]  # initial state
xf = @SVector [0, π, 0, 0.]  # goal state (i.e. swing up)

# Objective
Q = Diagonal(@SVector fill(1e-2,n))
R = Diagonal(@SVector fill(1e-1,m))
Qf = Diagonal(@SVector fill(100.,n))
obj = LQRObjective(Q, R, Qf, xf, N)

# Constraints
conSet = ConstraintList(n,m,N)
bnd = BoundConstraint(n,m, u_min=-3.0, u_max=3.0)
goal = GoalConstraint(xf)
add_constraint!(conSet, bnd, 1:N-1)
add_constraint!(conSet, goal, N:N)

In [5]:
prob = Problem(model, obj, x0, tf, constraints=conSet, xf=xf, integration=RobotDynamics.RK4(model))


Problem{Float64}(RobotDynamics.DiscretizedDynamics{Cartpole{Float64, 5}, RobotDynamics.RK4, 5}[RobotDynamics.DiscretizedDynamics{Cartpole{Float64, 5}, RobotDynamics.RK4, 5}(Cartpole{Float64, 5}(1.0, 0.2, 0.5, 9.81, ForwardDiff.JacobianConfig{Nothing, Float64, 5, Tuple{Vector{ForwardDiff.Dual{Nothing, Float64, 5}}, Vector{ForwardDiff.Dual{Nothing, Float64, 5}}}}((Partials(1.0, 0.0, 0.0, 0.0, 0.0), Partials(0.0, 1.0, 0.0, 0.0, 0.0), Partials(0.0, 0.0, 1.0, 0.0, 0.0), Partials(0.0, 0.0, 0.0, 1.0, 0.0), Partials(0.0, 0.0, 0.0, 0.0, 1.0)), (ForwardDiff.Dual{Nothing, Float64, 5}[Dual{Nothing}(2.1825811125e-314,2.1825811125e-314,2.1825811125e-314,2.1825811125e-314,2.1825811125e-314,2.1825811125e-314), Dual{Nothing}(2.1825811125e-314,2.1825811125e-314,2.1825811125e-314,2.1825811125e-314,2.1825811125e-314,2.1825811125e-314), Dual{Nothing}(2.1825811125e-314,2.1825811125e-314,2.1825811125e-314,2.1825811125e-314,2.1825811125e-314,2.1825811125e-314), Dual{Nothing}(2.1825811125e-314,2.1825811125e-31